In [3]:
import osmnx as ox
import networkx as nx
import geopandas as gpd
from ipyleaflet import Map, Marker, Polyline, LayerGroup
from IPython.display import display
import time

# Define the starting and ending locations
start_location = (30.618778910945345, -96.33762402554211)  # Example start location
end_location = (30.705851724745518, -96.46465632258864)    # Example end location

# Get the graph for the area around the starting location
G = ox.graph_from_point(start_location, dist=25000, network_type='drive')

# Get the nearest nodes to the starting and ending locations
start_node = ox.distance.nearest_nodes(G, X=start_location[1], Y=start_location[0])
end_node = ox.distance.nearest_nodes(G, X=end_location[1], Y=end_location[0])

# Use the A* algorithm to find the shortest path
route = nx.shortest_path(G, start_node, end_node, weight='length')

# Convert graph to GeoDataFrames
gdf_nodes, gdf_edges = ox.graph_to_gdfs(G)

# Extract the route nodes as a GeoDataFrame
route_nodes = gdf_nodes.loc[route]

# Create an interactive map
m = Map(center=start_location, zoom=12)

# Add start and end markers
start_marker = Marker(location=start_location, draggable=False, title='Start')
end_marker = Marker(location=end_location, draggable=False, title='End')
m.add_layer(start_marker)
m.add_layer(end_marker)

# Function to update the route on the map
def update_route(route_nodes):
    route_layer = LayerGroup()
    for i in range(len(route_nodes) - 1):
        point1 = (route_nodes.iloc[i].y, route_nodes.iloc[i].x)
        point2 = (route_nodes.iloc[i + 1].y, route_nodes.iloc[i + 1].x)
        line = Polyline(locations=[point1, point2], color="blue", fill=False)
        route_layer.add_layer(line)
    return route_layer

# Display the map
print(f'displaying map')
display(m)

# Initialize route layer
current_route_layer = None

# Update the route in real-time
for i in range(len(route_nodes) - 1):
    # Remove the previous route layer if it exists
    if current_route_layer:
        m.remove_layer(current_route_layer)
    
    # Create and add the new route layer
    current_route_layer = update_route(route_nodes.iloc[:i + 2])
    m.add_layer(current_route_layer)
    
    # Sleep to create an animation effect
    time.sleep(0.2)

print("Animation completed.")


displaying map


Map(center=[30.618778910945345, -96.33762402554211], controls=(ZoomControl(options=['position', 'zoom_in_text'…

Animation completed.
